# ZeroDivisionError is introducing error
And we're all too lazy to see it for what it is: **infinity** (**∞**)

- Are there solutions we cannot find because we're not using symbolic algebra systems (CAS: Computer Algebra Systems)?
- Also: What should we understand about precision?
  - ``Decimal('0.3') != Decimal(0.3)``

In [ ]:
from __future__ import division
import collections
import itertools
from decimal import Decimal

import pandas as pd
from sympy import init_printing, limit, oo
from sympy.abc import x
init_printing(use_unicode=True)

## range functions

In [2]:
#seq = range(-1,-0.1,0.1) + range(0.1,1,0.1)
# import numpy as np
# np.arange(-1, 1, 0.1)


def irange_(start, stop, step=1):
    cur = start
    step = step
    while cur <= stop:
        yield cur
        cur += step


def irange(start, stop, step=1):
    cur = start
    step = step
    i = 1
    while cur <= stop:
        yield cur
        cur = start + i*step
        i += 1


def drange_(start, stop, step=1):
    cur = Decimal(start)
    step = Decimal(step)
    while cur <= stop:
        yield cur
        cur += step


def drange(start, stop, step=1):
    cur = Decimal(str(start))  # note the cast to str first
    step = Decimal(str(step))  # note the cast to str first
    while cur <= stop:
        yield cur
        cur += step


def frange(start, stop, step=1):
    cur = float(start)
    step = step
    while cur <= stop:
        yield cur
        cur += step

In [3]:
import unittest

class Test_drange(unittest.TestCase):

    def test_drange(self):
        output = list(drange(-1,1, 0.5))
        self.assertEqual(output, [-1, -0.5, 0, 0.5, 1])

    def _TODO_FIXME_test_drange_2(self):
        # TODO FIXME XXX
        output = list(drange(-1,1, 0.3))
        expected = [-1, -0.7, -0.4, -0.1, 0.2, 0.5, 0.8,]
        self.assertAlmostEqual(output, expected)
        self.assertEqual(output, [Decimal(x) for x in expected])
        self.assertEqual(output, expected)
        
# unittest.main() # TODO: nbval?

## f(x)=1/x and ZeroDvisionError

In [4]:
def f(x):
    try:
        return 1 / x
    except ZeroDivisionError:
        #return 'ZeroDivisionError'
        return float('inf')
    except TypeError:
        if x is None:
            return None
        raise


def my_point_was():
    seq = drange(-1, 1, 0.1)
    # seq = frange(-1, 1, 0.1)
    #seq = irange_(-1, 1, 0.1)
    for x in seq:
        if x != 0:
            y = f(x)
            print((x, y))
        else:
            print((x, 'infinity (ZeroDivisionError)'))
my_point_was()

(Decimal('-1'), Decimal('-1'))
(Decimal('-0.9'), Decimal('-1.111111111111111111111111111'))
(Decimal('-0.8'), Decimal('-1.25'))
(Decimal('-0.7'), Decimal('-1.428571428571428571428571429'))
(Decimal('-0.6'), Decimal('-1.666666666666666666666666667'))
(Decimal('-0.5'), Decimal('-2'))
(Decimal('-0.4'), Decimal('-2.5'))
(Decimal('-0.3'), Decimal('-3.333333333333333333333333333'))
(Decimal('-0.2'), Decimal('-5'))
(Decimal('-0.1'), Decimal('-1E+1'))
(Decimal('0.0'), 'infinity (ZeroDivisionError)')
(Decimal('0.1'), Decimal('1E+1'))
(Decimal('0.2'), Decimal('5'))
(Decimal('0.3'), Decimal('3.333333333333333333333333333'))
(Decimal('0.4'), Decimal('2.5'))
(Decimal('0.5'), Decimal('2'))
(Decimal('0.6'), Decimal('1.666666666666666666666666667'))
(Decimal('0.7'), Decimal('1.428571428571428571428571429'))
(Decimal('0.8'), Decimal('1.25'))
(Decimal('0.9'), Decimal('1.111111111111111111111111111'))
(Decimal('1.0'), Decimal('1'))


## sympy.limit(1/x, x, 0, '+'/'-')

In [5]:
def sympy_says():
    """what is the limit of f(x)=1/x as x approaches 0? oo"""
    expr = (1/x)
    limit_positive = limit(expr, x, 0, '+')
    limit_negative = limit(expr, x, 0, '-')
    print(expr)
    print(limit_positive)
    print(limit_negative)

    print(10*oo / 2*oo)   # if oo is just a symbol, this is 5*oo
    print((10*oo / 2*oo) / oo)   # if oo is just a symbol, this is 5
sympy_says()

1/x
oo
-oo
oo
nan


## Comparing the precision of ints, floats, and Decimals

In [6]:
def thing(start, stop, step=1):
    args = (start, stop, step)
    funcs = [irange, irange_, frange, drange, drange_]
    data = collections.OrderedDict(
        ((func.__name__, func(*args)) for func in funcs))
    # print(data)

    df = pd.DataFrame.from_records(
        itertools.zip_longest(*data.values()),
        columns=data.keys())
    pd.set_option('expand_frame_repr', False)
    print(df)
    pd.set_option('expand_frame_repr', True)
    print(df.T)

    for func in funcs:
        name = func.__name__
        df['f(%s)' % name] = df[name].apply(f)
    print(df)
    print(df.T)  # TODO: why is this skipping 10?

    def to_decimal(n):
        return Decimal(str(n))

    for f1, f2 in itertools.combinations(funcs, 2):
        f1_name, f2_name = 'f(%s)' % f1.__name__, 'f(%s)' % f2.__name__
        name = "%s-%s" % (f1_name, f2_name)
        print(('name', name))
        # print(df[f1_name])
        if 'drange' in f1_name:
            #print(df[f2_name].astype(Decimal))
            df[name] = df[f1_name] - df[f2_name].apply(to_decimal)
        elif 'drange' in f2_name:
            #print(df[f1_name].astype(Decimal))
            df[name] = df[f1_name].apply(to_decimal) - df[f2_name]
        else:
            df[name] = df[f1_name] - df[f2_name]

    print(df)
    print(df.T)
    return df

In [7]:
def main():
    my_point_was()
    print('---')
    sympy_says()
    print('----')
    return thing(-1, 1, 0.1)
main()

(Decimal('-1'), Decimal('-1'))
(Decimal('-0.9'), Decimal('-1.111111111111111111111111111'))
(Decimal('-0.8'), Decimal('-1.25'))
(Decimal('-0.7'), Decimal('-1.428571428571428571428571429'))
(Decimal('-0.6'), Decimal('-1.666666666666666666666666667'))
(Decimal('-0.5'), Decimal('-2'))
(Decimal('-0.4'), Decimal('-2.5'))
(Decimal('-0.3'), Decimal('-3.333333333333333333333333333'))
(Decimal('-0.2'), Decimal('-5'))
(Decimal('-0.1'), Decimal('-1E+1'))
(Decimal('0.0'), 'infinity (ZeroDivisionError)')
(Decimal('0.1'), Decimal('1E+1'))
(Decimal('0.2'), Decimal('5'))
(Decimal('0.3'), Decimal('3.333333333333333333333333333'))
(Decimal('0.4'), Decimal('2.5'))
(Decimal('0.5'), Decimal('2'))
(Decimal('0.6'), Decimal('1.666666666666666666666666667'))
(Decimal('0.7'), Decimal('1.428571428571428571428571429'))
(Decimal('0.8'), Decimal('1.25'))
(Decimal('0.9'), Decimal('1.111111111111111111111111111'))
(Decimal('1.0'), Decimal('1'))
---
1/x
oo
-oo
oo
nan
----
    irange       irange_        frange drange 

TypeError: unsupported operand type(s) for -: 'decimal.Decimal' and 'float'